# Transform Coding

* Signals can be represented at least in two different domains: the signal domain (for example, time in the case of sound of space in the case of image) and the frequency domain.
* Transform coding is based on the idea that, in the frequency domain, most of the energy of the signal can be compacted in a small number of transform coefficients, which can be interesting for increasing the coding efficiency.

## Encoder

1. Split $s$ into blocks of $B$ samples.
2. Transform each block.
3. Performs a bit-allocation procedure over the coefficientes.
4. Encode the quantized coefficients.

## Decoder

1. Decode the coefficients of each block
2. "Dequantize" the coefficients of each block.
3. Inverse transform each block.

## Splitting

1. $$s=\{s[n]\}_{n=0}^{N-1}$$ is divided into blocks of $B$ samples.

* It can be shown that we can get the maximum amount
of compaction if we use a transform that decorrelates the input sequence; that is, the sample-
to-sample correlation of the transformed sequence is zero. 

* Usually lossy.
* If lossless, base (lossy) layer (a thumbnail sketch, potentially progressive) with the output of the transform encoder + enhacement (lossless) layer with the residue.
* Usually orthonormal transform.

* Most audio compression algorithms typically segment
the input signal into blocks of 2ms up to 50ms
duration. A time-frequency analysis then decomposes
each analysis block in the encoder. This
transformation or subband filtering scheme compacts
the energy into a few transform coefficients and
therefore de-correlates successive samples. These
coefficients, subband samples or parameters are
quantized and encoded according to perceptual
criteria. Depending on the system objectives, the
time-frequency analysis section might contain:
  + Unitary transform (MDCT, FFT)
  + Polyphase filterbank with uniform bandpass filters
  + Time-varying, critically sampled bank of nonuniform bandpass filters
  + Hybrid transform/filterbank scheme
  + Harmonic/sinusoidal signal analyzer
  + Source system analysis (LPC)
  

 
* The time-frequency analysis approach always
involves a fundamental tradeoff between time and
frequency resolution requirements. The choice of the
time-frequency analysis method additionally
determines the amount of coding delay introduced, a 
parameter which may become important in duplex
broadcast and live-events applications.

http://www.atc-labs.com/acepages/crcchapter.pdf

In [ ]:
The use of the cosine transform is motivated by the well known 
energy compactation energy, which permits a good reconstruction
from only a few quantized values of the cosine transform